In [1]:
from sympy import Matrix, Rational, nsimplify, pprint
import numpy as np

In [21]:
def to_sympy_matrix(mat):
    """
    Convertit une liste de listes (matrice) en sympy.Matrix.
    Si la matrice contient des floats, on essaie de les convertir en rationnels
    (nsimplify) pour un résultat exact quand c'est possible.
    """
    has_float = any(isinstance(x, float) for row in mat for x in row)
    if has_float:
        sym_rows = []
        for row in mat:
            sym_row = [nsimplify(x) for x in row]
            sym_rows.append(sym_row)
        return Matrix(sym_rows)
    else:
        return Matrix(mat)

def jordan_reduction(mat):
    """
    mat : list of lists or sympy.Matrix
    Retourne (P, J) où J = P^{-1} * A * P est la forme de Jordan, P la matrice de passage.
    """
    if not isinstance(mat, Matrix):
        A = to_sympy_matrix(mat)
    else:
        A = mat
    try:
        P, J = A.jordan_form()
    except Exception:
        # fallback si l'ordre est inversé dans cette version : essayer l'inverse
        out = A.jordan_form()
        if len(out) == 2:
            # essayer les deux ordres
            if A == out[0].inv() * out[1] * out[0]:
                P, J = out
            elif A == out[1].inv() * out[0] * out[1]:
                P, J = out[1], out[0]
            else:
                # dernière tentative : assigner (P,J) de façon "classique"
                P, J = out
        else:
            raise

    # vérification
    verified = (P.inv() * A * P).simplify() == J.simplify()
    return A, P, J, verified

if __name__ == "__main__":
    A_np = np.array([
    [4 , 3 , -2],
    [-3 , -1 , 3],
    [2 , 3 , 0]
])

    A = Matrix(A_np)
    P, J = A.jordan_form()
    
    print("Matrice A :")
    pprint(A)
    print("\nles valeur propre de A et leur multiplicité sont: ",A.eigenvals())
    print("\nMatrice de passage P :")
    pprint(P)
    print("\nForme de Jordan J :")
    pprint(J)
    print("\nVérification P^{-1} A P == J :", (P.inv()*A*P == J))

Matrice A :
⎡4   3   -2⎤
⎢          ⎥
⎢-3  -1  3 ⎥
⎢          ⎥
⎣2   3   0 ⎦

les valeur propre de A et leur multiplicité sont:  {-1: 1, 2: 2}

Matrice de passage P :
⎡1   1  -1⎤
⎢         ⎥
⎢-1  0  1 ⎥
⎢         ⎥
⎣1   1  0 ⎦

Forme de Jordan J :
⎡-1  0  0⎤
⎢        ⎥
⎢0   2  1⎥
⎢        ⎥
⎣0   0  2⎦

Vérification P^{-1} A P == J : True
